# Exercise 3 - Creating Useful Visualizations and Consistent Reports

In this exercise, we will focus on creating nice visualizations that can be used in reports. We will also show how the code you develop here can be compressed into a small file that you can run at regular intervals to update the reports (e.g., to download a map every day or every week).

We will first look at time series data, and then add a map. This would be useful to chart the amount of rainfall over a certain time period, as well as visualize it on a map. 

Let's start by adding some python modules:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
import cartopy
import cartopy.crs as ccrs
import datetime
from shapely.geometry import mapping

We can get data straight from Google Earth Engine, like we did earlier in the day:

In [ ]:
import ee
#The first time you use the earthengine module, you need to link your account credentials. Afterwards, your
#computer stores the authentication file

#ee.Authenticate()

ee.Initialize()
print(ee.__version__)

In [ ]:
dem = ee.Image('USGS/SRTMGL1_003') #Load in the global SRTM elevation data set
langtang = ee.Geometry.Point([85.51593, 28.21618]) 
elev = dem.sample(langtang, 30).first().get('elevation').getInfo() #Sample the data set at that point
print('Langtang elevation (m):', elev) #Print the result

In [ ]:
def create_data(data, variable, location):
    name = 'x'
    def create_time_series(data, variable, name):
        def create_(image):
            date = image.get('system:time_start')
            value = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=location).get(variable)
            ft = ee.Feature(None, {'date': ee.Date(date).format('Y/M/d-H:m:s'), name: value})
            return ft
        return data.map(create_).getInfo()
    
    time_series = create_time_series(data, variable, name)
    
    dates, datas = [], []
    for f in time_series['features']:
        properties = f['properties']
        date = properties['date']
        try:
            val = properties[name]
            datas.append(val)
            dates.append(datetime.datetime.strptime(date,'%Y/%m/%d-%H:%M:%S')) #Convert the date into something that Python recognizes
        except:
            pass
    return np.array(dates), np.array(datas)

In [ ]:
rainfall = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate('2022-01-01', '2024-03-01') #Last two years
rdates, rain = create_data(rainfall, 'precipitation', langtang)
print(rain)

## Plotting Time Series Data

Let's look a little bit more closely at how we make plots. There are many different options for making nice visualizations with python, depending on what kind of data you want to plot. We can of course first make a very basic plot with just the data:

In [ ]:
plt.plot(rdates, rain)

This works, but leaves the data floating in space. Let's start making some changes to the size of the chart, the labels, and the way that the data is symbolized:

In [ ]:
f, ax = plt.subplots(1, figsize=(10,5)) #This is where we control the size of the Figure we want to make
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

This already looks a bit clearer. Now let's add some lablels, and increase the size of the text:

In [ ]:
f, ax = plt.subplots(1, figsize=(10,5)) #This is where we control the size of the Figure we want to make
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

ax.set_xlabel('Date', fontsize=18, fontweight='bold') #Increase the size of the axis labels
ax.set_ylabel('Precipitation Sum (mm)', fontsize=18, fontweight='bold')

ax.tick_params(axis='both', which='major', labelsize=16) #Make the labels bigger
ax.tick_params(axis='x', which='major', rotation=45) #Rotate the x-labels so they are clearer

Let's also add markers on the actual rainfall points to make the lines easier to see:

In [ ]:
f, ax = plt.subplots(1, figsize=(10,5)) #This is where we control the size of the Figure we want to make
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

ax.set_xlabel('Date', fontsize=18, fontweight='bold') #Increase the size of the axis labels
ax.set_ylabel('Precipitation Sum (mm)', fontsize=18, fontweight='bold')

ax.tick_params(axis='both', which='major', labelsize=16) #Make the labels bigger
ax.tick_params(axis='x', which='major', rotation=45) #Rotate the x-labels so they are clearer

ax.plot(rdates[rain != 0], rain[rain != 0], 'rx', markersize=8, label='Rainfall') #Only plot non-zero days
ax.legend() #Add a legend for easy reference

Finally, let's add a title and gridlines:

In [ ]:
f, ax = plt.subplots(1, figsize=(10,5)) #This is where we control the size of the Figure we want to make
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

ax.set_xlabel('Date', fontsize=18, fontweight='bold') #Increase the size of the axis labels
ax.set_ylabel('Precipitation Sum (mm)', fontsize=18, fontweight='bold')

ax.tick_params(axis='both', which='major', labelsize=16) #Make the labels bigger
ax.tick_params(axis='x', which='major', rotation=45) #Rotate the x-labels so they are clearer

ax.plot(rdates[rain != 0], rain[rain != 0], 'rx', markersize=8, label='Rainfall') #Only plot non-zero days
ax.legend() #Add a legend for easy reference

ax.set_title('Langtang Rainfall, 2022-2024', fontsize=20, fontweight='bold') #Add a title
ax.grid(True) #Add a grid

If I compare this to my original image, this is much clearer! The first image would not be very helpful in a report, as it doesn't give a lot of information directly. However, the second one clearly displays the rainfall data.

In [ ]:
plt.plot(rdates, rain)

## Plotting Geographic Data

Let's use the same CHIRPS data to get a grid over our study area. We can quickly make an annual composite for 2023 over the Trishuli area:

In [ ]:
def run_export(image, crs, filename, scale, region, maxPixels=1e12, cloud_optimized=True):
    task_config = {'fileNamePrefix': filename,'crs': crs,'scale': scale,'maxPixels': maxPixels, 'fileFormat': 'GeoTIFF', 'formatOptions': {'cloudOptimized': cloud_optimized}, 'region': region,}
    task = ee.batch.Export.image.toDrive(image, filename, **task_config)
    task.start()

rainfall = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate('2023-01-01', '2023-12-31') #2023
rainfall_sum = rainfall.reduce(ee.Reducer.sum())

trishuli_outline = gpd.read_file('GeoData/Trishuli.geojson')
area_of_interest = ee.Geometry.MultiPolygon(ee.List(mapping(trishuli_outline.geometry[0])['coordinates']))
#run_export(rainfall_sum, 'epsg:4326', 'CHIRPS_2023_RainfallSum', scale=5566, region=area_of_interest)

To read the image data into Python, we will use the _rasterio_ library:

In [ ]:
#from osgeo import gdal
import rasterio
src = rasterio.open('GeoData/CHIRPS_2023_RainfallSum.tif')
grid_data = src.read(1)

In [ ]:
plt.imshow(grid_data)

That is our rainfall grid for the area around Trishuli -- not very helpful when we just visualize it like that! We need to add some geographic context to make sure our visualization is useful. 

Let's start by making a geographic axis. We can add coastlines to make sure things are working properly:

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.coastlines()

Now we want to plot both our image data (of rainfall) and our watershed outline to make it easier to see what we are looking at:

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax)

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax)

left, bottom, right, top = src.bounds #Get the geographic boundaries of our grid data
ax.set_extent([left - 0.1, right + 0.1, bottom - 0.1, top + 0.1], crs=ccrs.PlateCarree())
ax.imshow(grid_data, extent=(left, right, bottom, top))

#Add in a grid so we can locate ourselves
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.top_labels = False
gl.right_labels = False

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)

left, bottom, right, top = src.bounds #Get the geographic boundaries of our grid data
ax.set_extent([left - 0.1, right + 0.1, bottom - 0.1, top + 0.1], crs=ccrs.PlateCarree())
ax.imshow(grid_data, extent=(left, right, bottom, top))

#Add in a grid so we can locate ourselves
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.top_labels = False
gl.right_labels = False

This is looking pretty good! We want to add one more piece, however -- a color legend so we know what the data we are looking at represents. 

In [ ]:
f = plt.figure(figsize=(10,10))
ax = f.add_subplot(111, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)

left, bottom, right, top = src.bounds #Get the geographic boundaries of our grid data
ax.set_extent([left - 0.1, right + 0.1, bottom - 0.1, top + 0.1], crs=ccrs.PlateCarree())

#Add in the colormap
color = ax.imshow(grid_data, extent=(left, right, bottom, top))
cbar = plt.colorbar(color, ax=ax)
cbar.set_label('2023 Precipitation Sum (mm)', fontsize=18, fontweight='bold')

#Add in a grid so we can locate ourselves
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.top_labels = False
gl.right_labels = False

ax.set_title('2023 Precipitation Sum, Trishuli', fontsize=24, fontweight='bold')

This looks pretty nice! We could export some data and make a nice figure with it in a rather short time. 

## Combining Both Data Types

Let's combine both of these approaches (map and chart) into one small package. I will change the time series to reflect the precipitation sum for the whole Trishuli, so we get a nice picture of the distribution of rainfall in both time (chart) and space (map).

In [ ]:
rainfall = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate('2023-01-01', '2023-12-31')
rdates, rain = create_data(rainfall, 'precipitation', area_of_interest)

In [ ]:
f = plt.figure(figsize=(25,10))

#First do the map:
ax = f.add_subplot(121, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)

left, bottom, right, top = src.bounds #Get the geographic boundaries of our grid data
ax.set_extent([left - 0.1, right + 0.1, bottom - 0.1, top + 0.1], crs=ccrs.PlateCarree())

#Add in the colormap
color = ax.imshow(grid_data, extent=(left, right, bottom, top))
cbar = plt.colorbar(color, ax=ax)
cbar.set_label('2023 Precipitation Sum (mm)', fontsize=18, fontweight='bold')

#Add in a grid so we can locate ourselves
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.top_labels = False
gl.right_labels = False

ax.set_title('2023 Precipitation Sum, Trishuli', fontsize=24, fontweight='bold')

#Then add the chart
ax = f.add_subplot(122)
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

ax.set_xlabel('Date', fontsize=18, fontweight='bold') #Increase the size of the axis labels
ax.set_ylabel('Daily Precipitation (mm)', fontsize=18, fontweight='bold')

ax.tick_params(axis='both', which='major', labelsize=16) #Make the labels bigger
ax.tick_params(axis='x', which='major', rotation=45) #Rotate the x-labels so they are clearer

ax.plot(rdates[rain != 0], rain[rain != 0], 'rx', markersize=8, label='Rainfall') #Only plot non-zero days
ax.legend() #Add a legend for easy reference

ax.set_title('Trishuli Rainfall, 2023', fontsize=20, fontweight='bold') #Add a title
ax.grid(True) #Add a grid

Now we have quite a nice little graphic to add to a report about rainfall patterns. It would be easy to modify this to use other data, or to have a longer time period -- you can use the same commands for downloading data and making these simple charts. 

## Making Consistent Reports

There are many reports that would be useful to get on a weekly basis -- for example, snow cover over the last week, recent rainfall, etc. It is possible to use the same basic code we have just developed in a short package that will save data for you. 

This is particularly easy to do with time series, since we can acquire the data directly. It is also possible to do with gridded data, if you can install Google Drive on your compuater. In this case, any files you make in Google Earth Engine would then be available on your computer for plotting.

Let's set up a simple example with high-frequency rainfall data over Trishuli. We will use GPM data [https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06#bands](https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06#bands), as it is low-latency -- we can get data from 24 hours ago. 

In [ ]:
def mask_(image):
    mask = image.gt(0.5)
    return image.updateMask(mask)

#This data is in mm/hr: https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06#bands
hr_rainfall = ee.ImageCollection("NASA/GPM_L3/IMERG_V06").select('precipitationCal').filterDate('2024-02-01', '2024-03-01')
hr_rainfall = hr_rainfall.map(mask_) #Remove low values

In [ ]:
#Create the time series
rdates, rain = create_data(hr_rainfall, 'precipitationCal', area_of_interest)

In [ ]:
#Create the grid
rain_sum = hr_rainfall.reduce(ee.Reducer.sum())
#run_export(rain_sum, 'epsg:4326', 'GPM_Feb2024_RainfallSum', scale=11132, region=area_of_interest)

In [ ]:
#Load the new data
import time, os
src = rasterio.open('GeoData/GPM_Feb2024_RainfallSum.tif')
#while not os.path.exists('/home/ttsmith/gdrive/GPM_Feb2024_RainfallSum.tif'):
#    time.sleep(10)
#src = rasterio.open('/home/ttsmith/gdrive/GPM_Feb2024_RainfallSum.tif')
grid_data = src.read(1)

#Make the plot
f = plt.figure(figsize=(25,10))

#First do the map:
ax = f.add_subplot(121, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)

left, bottom, right, top = src.bounds #Get the geographic boundaries of our grid data
ax.set_extent([left - 0.1, right + 0.1, bottom - 0.1, top + 0.1], crs=ccrs.PlateCarree())

#Add in the colormap
color = ax.imshow(grid_data, extent=(left, right, bottom, top))
cbar = plt.colorbar(color, ax=ax)
cbar.set_label('Feb 2024 Precipitation Sum (mm)', fontsize=18, fontweight='bold')

#Add in a grid so we can locate ourselves
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.top_labels = False
gl.right_labels = False

ax.set_title('Feb 2024 Precipitation Sum, Trishuli', fontsize=24, fontweight='bold')

#Then add the chart
ax = f.add_subplot(122)
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

ax.set_xlabel('Date', fontsize=18, fontweight='bold') #Increase the size of the axis labels
ax.set_ylabel('Daily Precipitation (mm)', fontsize=18, fontweight='bold')

ax.tick_params(axis='both', which='major', labelsize=16) #Make the labels bigger
ax.tick_params(axis='x', which='major', rotation=45) #Rotate the x-labels so they are clearer

ax.plot(rdates[rain != 0], rain[rain != 0], 'rx', markersize=8, label='Rainfall') #Only plot non-zero days
ax.legend() #Add a legend for easy reference

ax.set_title('Trishuli Rainfall, Feb 2024', fontsize=20, fontweight='bold') #Add a title
ax.grid(True) #Add a grid

Since my Google Drive is linked to my computer, I can run this directly. This means that I can create a new figure just by changing a couple of parameters. For example, let's just extend this backwards in time by an additional month:

In [ ]:
#Open the Google Earth Engine Data
#This data is in mm/hr: https://developers.google.com/earth-engine/datasets/catalog/NASA_GPM_L3_IMERG_V06#bands
hr_rainfall = ee.ImageCollection("NASA/GPM_L3/IMERG_V06").select('precipitationCal').filterDate('2024-01-01', '2024-03-01')
hr_rainfall = hr_rainfall.map(mask_) #Remove low values

#Create the time series
rdates, rain = create_data(hr_rainfall, 'precipitationCal', area_of_interest)

#Create the grid
rain_sum = hr_rainfall.reduce(ee.Reducer.sum())
run_export(rain_sum, 'epsg:4326', 'GPM_JanFeb2024_RainfallSum', scale=11132, region=area_of_interest)

#Load the new data
import time, os
src = rasterio.open('GeoData/GPM_JanFeb2024_RainfallSum.tif')
#while not os.path.exists('/home/ttsmith/gdrive/GPM_JanFeb2024_RainfallSum.tif'):
#    time.sleep(10)
#src = rasterio.open('/home/ttsmith/gdrive/GPM_JanFeb2024_RainfallSum.tif')
grid_data = src.read(1)

#Make the plot
f = plt.figure(figsize=(25,10))

#First do the map:
ax = f.add_subplot(121, projection=ccrs.PlateCarree()) #Create a geographic axis
ax.add_feature(cartopy.feature.BORDERS, linestyle='--')
trishuli_outline.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=2)

left, bottom, right, top = src.bounds #Get the geographic boundaries of our grid data
ax.set_extent([left - 0.1, right + 0.1, bottom - 0.1, top + 0.1], crs=ccrs.PlateCarree())

#Add in the colormap
color = ax.imshow(grid_data, extent=(left, right, bottom, top))
cbar = plt.colorbar(color, ax=ax)
cbar.set_label('Jan-Feb 2024 Precipitation Sum (mm)', fontsize=18, fontweight='bold')

#Add in a grid so we can locate ourselves
gl = ax.gridlines(crs=ccrs.PlateCarree(), linewidth=1, color='black', alpha=0.5, linestyle='--', draw_labels=True)
gl.top_labels = False
gl.right_labels = False

ax.set_title('Jan-Feb 2024 Precipitation Sum, Trishuli', fontsize=24, fontweight='bold')

#Then add the chart
ax = f.add_subplot(122)
ax.plot(rdates, rain)
ax.set_ylim(ymin=0) #Set the minimum y-value to 0
ax.set_xlim(rdates[0], rdates[-1]) #Make the plot only go to the first and last date in our data

ax.set_xlabel('Date', fontsize=18, fontweight='bold') #Increase the size of the axis labels
ax.set_ylabel('Daily Precipitation (mm)', fontsize=18, fontweight='bold')

ax.tick_params(axis='both', which='major', labelsize=16) #Make the labels bigger
ax.tick_params(axis='x', which='major', rotation=45) #Rotate the x-labels so they are clearer

ax.plot(rdates[rain != 0], rain[rain != 0], 'rx', markersize=8, label='Rainfall') #Only plot non-zero days
ax.legend() #Add a legend for easy reference

ax.set_title('Trishuli Rainfall, Jan-Feb 2024', fontsize=20, fontweight='bold') #Add a title
ax.grid(True) #Add a grid

## Further Information

Once you are happy with how your figure looks, you can just change one feature -- for example, the date range, or the chosen area of interest -- and create the same figure without a lot of work! This can be a really nice way to create consistent graphics for reports or research. 

On the workshop github, we have provided the same code as a python function that you can run directly. This will generate the same graphic we have created here, but without the extra commentary. 